# 1. Getting started
The purpose of this notebook is to provide the basic building blocks for working with the Dimensions DSL.

## A. Interacting with Dimensions using DimCli

[DimCli](https://github.com/lambdamusic/dimcli) is an open source python library which contains various commands that make it easier to interact with the Dimensions API from Python notebooks. 

> This guide assumes that you already have a Python 3 working environment and [pip](https://pypi.org/project/pip/) - the python package manager - installed. For more background, see this [link](https://realpython.com/installing-python/).

If you haven't installed it already, you can install DimCli do so as follows: 

In [ ]:
!pip install dimcli -U

Then each time you want to use it within a notebook you can load it like this:

In [ ]:
import dimcli
%dsl_login 

### Authentication

DimCli handles authenticating with the Dimensions API. This requires you to store your access credentials (email and password most likely) in a file on your computer so that DimCli can retrieve those data at runtime.

There are [different ways](https://github.com/lambdamusic/dimcli#the-credentials-file) to achieve that, but the easiest is to add a file called `dsl.ini` in the current working directory (eg where your notebooks are located).

The file contents should follow this structure:

```
[instance.live]
url=https://app.dimensions.ai
login=user@mail.com
password=yourpasswordhere
```

You just need to update the login and password fields with your credentials.

### Querying

DimCli includes a couple of [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) which make querying very easy: `dsl_query` and `dsl_query_loop`.

##### Magic Command: Dsl_Query

In [ ]:
%dsl_query search publications where journal.title="Nature" return publications

Or using all the space in a cell (note the `%%` instead of `%`)

In [ ]:
%%dsl_query
search publications
where year in [2013:2018] and research_orgs="grid.258806.1"
return publications[title]

Queries return a DimCli `Result` object, which is just a wrapper around the raw JSON data. 

We can assign the results of the magic command to a variable, for example, and explore the returned data:

In [ ]:
res = %dsl_query search publications where journal.title="Nature" return publications

In [ ]:
type(res)

In [ ]:
res['publications'][2]

##### Magic Command: Dsl_query_loop

The second command you may want to use is `dsl_query_loop`. This is similar to the one above, but it loops over all the pages of a result set until all results can be returned.

It is useful to remember that the Dimensions DSL returns a maximum of 1000 records per query. So if your query returns 5000 records the only way to pull them out is to iterate through the results using the `limit` and `skip` operators (see the [docs](https://docs.dimensions.ai/dsl/language.html#limit-and-skip)). For simple queries with only one return statement, `dsl_query_loop` can do the looping for you in the background and return directly a `Result` object you can work this.

> NOTE: loop queries must be used with caution as they can result in a large number of API calls

In [ ]:
%%dsl_query_loop
search publications
where year in [2013:2018] and research_orgs="grid.258806.1"
return publications

### Other commands

> note: this section contains functionalities still being developed

The `%dsl_schema` is a shortcut for the dsl `describe schema` command, which outputs the dsl grammar as a JSON dictionary.

In [ ]:
%dsl_schema

## B. Understanding query results

When performing a DSL search, a `_stats` object is return which contains some useful info eg the total number of records available for a search. 

#### What the statistics refer to

It is important to note though that the total number *always refers to the main source* one is searching for, not necessarily the results being returned. 

So, both the queries below have `publications` as the main source, while only the first one also has `publications` in the return data. The second one instead returns `researchers`. 

In [ ]:
%%dsl_query
search publications
where year in [2013:2018] and research_orgs="grid.258806.1"
return publications

In [ ]:
%%dsl_query
search publications
where year in [2013:2018] and research_orgs="grid.258806.1"
return researchers

> note: the total number of publications in the queries above don't match because they are an _approximation_ of the real number of records available. This is due to the fact that in order to obtain adequate performance, the Dimensions database subdivides the data into multiple copies (or _shards_) which are not always perfectly synced. Hence the difference. 

#### Facets VS Sources
The previous query is using the `researchers` facet of the `publications` source. 

In general queries can return up to 1000 records. For example this throws an exception:

In [ ]:
%%dsl_query
search publications
where year in [2013:2018] and research_orgs="grid.258806.1"
return researchers limit 2000

Now, with facets there is no way to pull out more than 1000 records with one single query. Instead if you are returning sources we can use the `skip` keyword to paginate through the results. E.g. if we return `publications` we can do this:

In [ ]:
%%dsl_query
search publications
where year in [2013:2018] and research_orgs="grid.258806.1"
return publications limit 1000 skip 1000